<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/banner.png" align="left" alt="banner">

# Working with Watson Machine Learning

This notebook should be run in a Watson Studio project, using **Default Python 3.6** runtime environment. It requires service credentials for the following Cloud services:
  * Watson OpenScale
  * Watson Machine Learning
  
If you have a paid Cloud account, you may also provision a **Databases for PostgreSQL** or **Db2 Warehouse** service to take full advantage of integration with Watson Studio and continuous learning services. If you choose not to provision this paid service, you can use the free internal PostgreSQL storage with OpenScale, but will not be able to configure continuous learning for your model.

The notebook will train, create and deploy a German Credit Risk model, configure OpenScale to monitor that deployment, and inject seven days' worth of historical records and measurements for viewing in the OpenScale Insights dashboard.

### Contents

- [Setup](#setup)
- [Model building and deployment](#model)
- [OpenScale configuration](#openscale)
- [Quality monitor and feedback logging](#quality)
- [Fairness monitoring, Drift and explanations](#fairness)
- [Custom monitors and metrics](#custom)
- [Historical data](#historical)

### Note: Sample using most latest openscale V2 client from test pypi and will be changed to production release once available. It does not cover followings aspects for now:

- Historical payload logging 
- Historical manual labeling

# Setup <a name="setup"></a>

## Package installation

In [1]:
!rm -rf /home/spark/shared/user-libs/python3.6*

!pip install --upgrade pandas==0.25.3 --no-cache | tail -n 1
!pip install --upgrade requests==2.23 --no-cache | tail -n 1
!pip install --upgrade numpy==1.16.4 --no-cache | tail -n 1
!pip install --upgrade SciPy --no-cache | tail -n 1
!pip install lime --no-cache | tail -n 1
!pip install pixiedust | tail -n 1

!pip install ibm-cloud-sdk-core --no-cache | tail -n 1
!pip install ibm-watson-openscale --extra-index-url https://test.pypi.org/simple/ --no-cache | tail -n 1
#!pip install ibm-watson-openscale --no-cache | tail -n 1
!pip install --upgrade watson-machine-learning-client --user | tail -n 1

ERROR: tensorflow 1.13.1 requires tensorboard<1.14.0,>=1.13.0, which is not installed.


## Provision services and configure credentials

If you have not already, provision an instance of IBM Watson OpenScale using the [OpenScale link in the Cloud catalog](https://cloud.ibm.com/catalog/services/watson-openscale).

Your Cloud API key can be generated by going to the [**Users** section of the Cloud console](https://cloud.ibm.com/iam#/users). From that page, click your name, scroll down to the **API Keys** section, and click **Create an IBM Cloud API key**. Give your key a name and click **Create**, then copy the created key and paste it below.

**NOTE:** You can also get OpenScale `API_KEY` using IBM CLOUD CLI.

How to install IBM Cloud (bluemix) console: [instruction](https://console.bluemix.net/docs/cli/reference/ibmcloud/download_cli.html#install_use)

How to get api key using console:
```
bx login --sso
bx iam api-key-create 'my_key'
```

In [2]:
CLOUD_API_KEY = ""

Next you will need credentials for Watson Machine Learning. If you already have a WML instance, you may use credentials for it. To provision a new Lite instance of WML, use the [Cloud catalog](https://cloud.ibm.com/catalog/services/machine-learning), give your service a name, and click **Create**. Once your instance is created, click the **Service Credentials** link on the left side of the screen. Click the **New credential** button, give your credentials a name, and click **Add**. Your new credentials can be accessed by clicking the **View credentials** button. Copy and paste your WML credentials into the cell below.

In [3]:
WML_CREDENTIALS = {
  
}

This tutorial can use Databases for PostgreSQL, Db2 Warehouse, or a free internal verison of PostgreSQL to create a datamart for OpenScale.

If you have previously configured OpenScale, it will use your existing datamart, and not interfere with any models you are currently monitoring. Do not update the cell below.

If you do not have a paid Cloud account or would prefer not to provision this paid service, you may use the free internal PostgreSQL service with OpenScale. Do not update the cell below.

To provision a new instance of Db2 Warehouse, locate [Db2 Warehouse in the Cloud catalog](https://cloud.ibm.com/catalog/services/db2-warehouse), give your service a name, and click **Create**. Once your instance is created, click the **Service Credentials** link on the left side of the screen. Click the **New credential** button, give your credentials a name, and click **Add**. Your new credentials can be accessed by clicking the **View credentials** button. Copy and paste your Db2 Warehouse credentials into the cell below.

To provision a new instance of Databases for PostgreSQL, locate [Databases for PostgreSQL in the Cloud catalog](https://cloud.ibm.com/catalog/services/databases-for-postgresql), give your service a name, and click **Create**. Once your instance is created, click the **Service Credentials** link on the left side of the screen. Click the **New credential** button, give your credentials a name, and click **Add**. Your new credentials can be accessed by clicking the **View credentials** button. Copy and paste your Databases for PostgreSQL credentials into the cell below.

In [4]:
DB_CREDENTIALS = None
SCHEMA_NAME = None

__If you previously configured OpenScale to use the free internal version of PostgreSQL, you can switch to a new datamart using a paid database service.__ If you would like to delete the internal PostgreSQL configuration and create a new one using service credentials supplied in the cell above, set the __KEEP_MY_INTERNAL_POSTGRES__ variable below to __False__ below. In this case, the notebook will remove your existing internal PostgreSQL datamart and create a new one with the supplied credentials. __*NO DATA MIGRATION WILL OCCUR.*__

In [5]:
KEEP_MY_INTERNAL_POSTGRES = True

In next cells, you will need to paste some credentials to Cloud Object Storage. If you haven't worked with COS yet please visit [getting started with COS tutorial](https://cloud.ibm.com/docs/cloud-object-storage?topic=cloud-object-storage-getting-started). 
You can find `COS_API_KEY_ID` and `COS_RESOURCE_CRN` variables in **_Service Credentials_** in menu of your COS instance. Used COS Service Credentials must be created with _Role_ parameter set as Writer. Later training data file will be loaded to the bucket of your instance and used as training refecence in subsription.  
`COS_ENDPOINT` variable can be found in **_Endpoint_** field of the menu.

In [6]:
COS_API_KEY_ID = "***"
COS_RESOURCE_CRN = "***" # eg "crn:v1:bluemix:public:cloud-object-storage:global:a/3bf0d9003abfb5d29761c3e97696b71c:d6f04d83-6c4f-4a62-a165-696756d63903::"
COS_ENDPOINT = "***" # Current list avaiable at https://control.cloud-object-storage.cloud.ibm.com/v2/endpoints

In [7]:
BUCKET_NAME = "***" #example: "credit-risk-training-data"

## Run the notebook

At this point, the notebook is ready to run. You can either run the cells one at a time, or click the **Kernel** option above and select **Restart and Run All** to run all the cells.

# Model building and deployment <a name="model"></a>

In this section you will learn how to train Scikit-learn model and next deploy it as web-service using Watson Machine Learning service.

## Load the training data from github

In [8]:
!rm german_credit_data_biased_training.csv
!wget https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/assets/historical_data/german_credit_risk/wml/german_credit_data_biased_training.csv

--2020-08-18 18:29:18--  https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/assets/historical_data/german_credit_risk/wml/german_credit_data_biased_training.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.232.8.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.8.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 689622 (673K) [text/plain]
Saving to: ‘german_credit_data_biased_training.csv’

100%[======================================>] 689,622     --.-K/s   in 0.02s   

2020-08-18 18:29:18 (28.6 MB/s) - ‘german_credit_data_biased_training.csv’ saved [689622/689622]



In [9]:
import numpy as np
import pandas as pd

training_data_file_name = "german_credit_data_biased_training.csv"
data_df = pd.read_csv(training_data_file_name)

## Explore data

In [10]:
data_df.head()

,CheckingStatus,LoanDuration,CreditHistory,LoanPurpose,LoanAmount,ExistingSavings,EmploymentDuration,InstallmentPercent,Sex,OthersOnLoan,...,OwnsProperty,Age,InstallmentPlans,Housing,ExistingCreditsCount,Job,Dependents,Telephone,ForeignWorker,Risk
0,0_to_200,31,credits_paid_to_date,other,1889,100_to_500,less_1,3,female,none,...,savings_insurance,32,none,own,1,skilled,1,none,yes,No Risk
1,less_0,18,credits_paid_to_date,car_new,462,less_100,1_to_4,2,female,none,...,savings_insurance,37,stores,own,2,skilled,1,none,yes,No Risk
2,less_0,15,prior_payments_delayed,furniture,250,less_100,1_to_4,2,male,none,...,real_estate,28,none,own,2,skilled,1,yes,no,No Risk
3,0_to_200,28,credits_paid_to_date,retraining,3693,less_100,greater_7,3,male,none,...,savings_insurance,32,none,own,1,skilled,1,none,yes,No Risk
4,no_checking,28,prior_payments_delayed,education,6235,500_to_1000,greater_7,3,male,none,...,unknown,57,none,own,2,skilled,1,none,yes,Risk


In [11]:
print('Columns: ', list(data_df.columns))
print('Number of columns: ', len(data_df.columns))

Columns:  ['CheckingStatus', 'LoanDuration', 'CreditHistory', 'LoanPurpose', 'LoanAmount', 'ExistingSavings', 'EmploymentDuration', 'InstallmentPercent', 'Sex', 'OthersOnLoan', 'CurrentResidenceDuration', 'OwnsProperty', 'Age', 'InstallmentPlans', 'Housing', 'ExistingCreditsCount', 'Job', 'Dependents', 'Telephone', 'ForeignWorker', 'Risk']
Number of columns:  21


As you can see, the data contains twenty one fields. `Risk` field is the one you would like to predict using feedback data.

In [12]:
print('Number of records: ', data_df.Risk.count())

Number of records:  5000


In [13]:
target_count = data_df.groupby('Risk')['Risk'].count()
target_count

Risk
No Risk    3330
Risk       1670
Name: Risk, dtype: int64

## Visualize data

In [14]:
target_count.plot.pie(figsize=(8, 8));

## Save training data to Cloud Object Storage

In [15]:
import ibm_boto3
from ibm_botocore.client import Config, ClientError

cos_client = ibm_boto3.resource("s3",
    ibm_api_key_id=COS_API_KEY_ID,
    ibm_service_instance_id=COS_RESOURCE_CRN,
    ibm_auth_endpoint="https://iam.bluemix.net/oidc/token",
    config=Config(signature_version="oauth"),
    endpoint_url=COS_ENDPOINT
)

In [16]:
with open(training_data_file_name, "rb") as file_data:
    cos_client.Object(BUCKET_NAME, training_data_file_name).upload_fileobj(
        Fileobj=file_data
    )

## Create a model
In this section you will learn how to:

- Prepare data for training a model
- Create machine learning pipeline
- Train a model

In [17]:
MODEL_NAME = "Scikit German Risk Model"
DEPLOYMENT_NAME = "Scikit German Risk Deployment"

### You will start with importing required libraries

In [18]:
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.decomposition import TruncatedSVD
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

### Splitting the data into train and test

In [19]:
train_data, test_data = train_test_split(data_df, test_size=0.2)

### Preparing the pipeline

In [20]:
features_idx = np.s_[0:-1]
all_records_idx = np.s_[:]
first_record_idx = np.s_[0]

In this step you will encode target column labels into numeric values. You can use `inverse_transform` to decode numeric predictions into labels.

In [21]:
string_fields = [type(fld) is str for fld in train_data.iloc[first_record_idx, features_idx]]
ct = ColumnTransformer([("ohe", OneHotEncoder(), list(np.array(train_data.columns)[features_idx][string_fields]))])
clf_linear = SGDClassifier(loss='log', penalty='l2', max_iter=1000, tol=1e-5)

pipeline_linear = Pipeline([('ct', ct), ('clf_linear', clf_linear)])

### Train a model

In [22]:
risk_model = pipeline_linear.fit(train_data.drop('Risk', axis=1), train_data.Risk)

### Evaluate the model

In [23]:
from sklearn.metrics import roc_auc_score

predictions = risk_model.predict(test_data.drop('Risk', axis=1))
indexed_preds = [0 if prediction=='No Risk' else 1 for prediction in predictions]

real_observations = test_data.Risk.replace('Risk', 1)
real_observations = real_observations.replace('No Risk', 0).values

auc = roc_auc_score(real_observations, indexed_preds)
print(auc)

0.7295948643254032


## Publish the model

In this section, the notebook uses the supplied Watson Machine Learning credentials to save the model (including the pipeline) to the WML instance. Previous versions of the model are removed so that the notebook can be run again, resetting all data for another demo.

In [24]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient
import json

wml_client = WatsonMachineLearningAPIClient(WML_CREDENTIALS)

### Remove existing model and deployment

In [25]:
model_deployment_ids = wml_client.deployments.get_uids()
for deployment_id in model_deployment_ids:
    deployment = wml_client.deployments.get_details(deployment_id)
    model_id = deployment["entity"]["deployable_asset"]["guid"]
    if deployment["entity"]["name"] == DEPLOYMENT_NAME:
        print("Deleting deployment id", deployment_id)
        wml_client.deployments.delete(deployment_id)
        print("Deleting model id", model_id)
        wml_client.repository.delete(model_id)
wml_client.repository.list_models()
del deployment_id

------------------------------------  -------------------------------  ------------------------  ---------
GUID                                  NAME                             CREATED                   FRAMEWORK
d4212921-bad9-42bd-861a-9754f584815d  Spark German Risk Model - Final  2020-08-18T17:18:14.014Z  mllib-2.3
91be60bd-c9af-4103-9732-4be6a336ae50  Income Classifier Binary Model   2020-07-23T17:58:45.824Z  mllib-2.3
------------------------------------  -------------------------------  ------------------------  ---------


In [26]:
model_props = {
    wml_client.repository.ModelMetaNames.NAME: "{}".format(MODEL_NAME),
    wml_client.repository.ModelMetaNames.EVALUATION_METHOD: "binary",
    wml_client.repository.ModelMetaNames.EVALUATION_METRICS: [
        {
           "name": "areaUnderROC",
           "value": auc,
           "threshold": 0.7
        }
    ]
}

In [27]:
wml_models = wml_client.repository.get_details()
model_uid = None
for model_in in wml_models["models"]["resources"]:
    if MODEL_NAME == model_in["entity"]["name"]:
        model_uid = model_in["metadata"]["guid"]
        break

if model_uid is None:
    print("Storing model ...")

    published_model_details = wml_client.repository.store_model(model=risk_model, meta_props=model_props, training_data=data_df.drop(["Risk"], axis=1), training_target=data_df.Risk)
    model_uid = wml_client.repository.get_model_uid(published_model_details)
    print("Done")

Storing model ...
Done


## Deploy the model

The next section of the notebook deploys the model as a RESTful web service in Watson Machine Learning. The deployed model will have a scoring URL you can use to send data to the model for predictions.

In [28]:
wml_deployments = wml_client.deployments.get_details()
deployment_uid = None
for deployment in wml_deployments["resources"]:
    if DEPLOYMENT_NAME == deployment["entity"]["name"]:
        deployment_uid = deployment["metadata"]["guid"]
        break

if deployment_uid is None:
    print("Deploying model...")

    deployment = wml_client.deployments.create(artifact_uid=model_uid, name=DEPLOYMENT_NAME, asynchronous=False)
    deployment_uid = wml_client.deployments.get_uid(deployment)
    
print("Model id: {}".format(model_uid))
print("Deployment id: {}".format(deployment_uid))

Deploying model...


#######################################################################################

Synchronous deployment creation for uid: 'c3fbb6ba-1dcb-4aa4-9073-2c0f32d7451c' started

#######################################################################################


INITIALIZING
DEPLOY_SUCCESS


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='d9df97da-38aa-4a2f-b4af-4daa0f4e1ff4'
------------------------------------------------------------------------------------------------


Model id: c3fbb6ba-1dcb-4aa4-9073-2c0f32d7451c
Deployment id: d9df97da-38aa-4a2f-b4af-4daa0f4e1ff4


## Score the model

In [29]:
fields = ["CheckingStatus", "LoanDuration", "CreditHistory", "LoanPurpose", "LoanAmount", "ExistingSavings",
                  "EmploymentDuration", "InstallmentPercent", "Sex", "OthersOnLoan", "CurrentResidenceDuration",
                  "OwnsProperty", "Age", "InstallmentPlans", "Housing", "ExistingCreditsCount", "Job", "Dependents",
                  "Telephone", "ForeignWorker"]
values = [
            ["no_checking", 13, "credits_paid_to_date", "car_new", 1343, "100_to_500", "1_to_4", 2, "female", "none", 3,
             "savings_insurance", 46, "none", "own", 2, "skilled", 1, "none", "yes"],
            ["no_checking", 24, "prior_payments_delayed", "furniture", 4567, "500_to_1000", "1_to_4", 4, "male", "none",
             4, "savings_insurance", 36, "none", "free", 2, "management_self-employed", 1, "none", "yes"],
        ]

scoring_payload = {"fields": fields, "values": values}

In [30]:
scoring_url = wml_client.deployments.get_scoring_url(deployment)

wml_client.deployments.score(scoring_url, scoring_payload)

{'fields': ['prediction', 'probability'],
 'values': [['No Risk', [0.6270374722208658, 0.3729625277791342]],
  ['No Risk', [0.7819593884550529, 0.21804061154494714]]]}

# Configure OpenScale <a name="openscale"></a>

The notebook will now import the necessary libraries and set up a Python OpenScale client.

In [31]:
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

from ibm_watson_openscale import *
from ibm_watson_openscale.supporting_classes.enums import *
from ibm_watson_openscale.supporting_classes import *

authenticator = IAMAuthenticator(apikey=CLOUD_API_KEY)
wos_client = APIClient(authenticator=authenticator, service_url="https://api.aiopenscale.test.cloud.ibm.com")
wos_client.version

'3.0.0.17'

### Get Watson OpenScale GUID

Each instance of OpenScale has a unique ID. We can get this value using the Cloud API key specified at the beginning of the notebook.
1. Please update the `url` in the below WOS_CREDENTIALS payload as per the environment that you are using.
2. Please update the `DASHBOARD_URL` in the below cell as per the environment that you are using.

In [ ]:
from ibm_watson_openscale.utils import get_instance_guid
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

authenticator = IAMAuthenticator(
                    apikey=CLOUD_API_KEY,
                    url=IAM_URL
                )
WOS_GUID = get_instance_guid(authenticator)

WOS_CREDENTIALS = {
    "instance_guid": WOS_GUID,
    "apikey": CLOUD_API_KEY,
    "url": "https://api.aiopenscale.cloud.ibm.com"
}
DASHBOARD_URL = "https://aiopenscale.cloud.ibm.com"


if WOS_GUID is None:
    print('Watson OpenScale GUID NOT FOUND')
else:
    print(WOS_GUID)

## Create schema and datamart

### Set up datamart

Watson OpenScale uses a database to store payload logs and calculated metrics. If database credentials were **not** supplied above, the notebook will use the free, internal lite database. If database credentials were supplied, the datamart will be created there **unless** there is an existing datamart **and** the **KEEP_MY_INTERNAL_POSTGRES** variable is set to **True**. If an OpenScale datamart exists in Db2 or PostgreSQL, the existing datamart will be used and no data will be overwritten.

Prior instances of the German Credit model will be removed from OpenScale monitoring.

In [32]:
wos_client.data_marts.show()

None,None,True,active,2020-07-20 18:06:32.485000+00:00,5a0b9076-fcf6-49e8-a824-9e3a6b4c2a56


In [33]:
data_marts = wos_client.data_marts.list().result.data_marts
if len(data_marts) == 0:
    if DB_CREDENTIALS is not None:
        if SCHEMA_NAME is None: 
            print("Please specify the SCHEMA_NAME and rerun the cell")

        print("Setting up external datamart")
        added_data_mart_result = wos_client.data_marts.add(
                background_mode=False,
                name="WOS Data Mart",
                description="Data Mart created by WOS tutorial notebook",
                database_configuration=DatabaseConfigurationRequest(
                  database_type=DatabaseType.POSTGRESQL,
                    credentials=PrimaryStorageCredentialsLong(
                        hostname=DB_CREDENTIALS["connection"]["postgres"]["hosts"][0]["hostname"],
                        username=DB_CREDENTIALS["connection"]["postgres"]["authentication"]["username"],
                        password=DB_CREDENTIALS["connection"]["postgres"]["authentication"]["password"],
                        db=DB_CREDENTIALS["connection"]["postgres"]["database"],
                        port=DB_CREDENTIALS["connection"]["postgres"]["hosts"][0]["port"],
                        ssl=True,
                        sslmode=DB_CREDENTIALS["connection"]["postgres"]["query_options"]["sslmode"],
                        certificate_base64=DB_CREDENTIALS["connection"]["postgres"]["certificate"]["certificate_base64"]
                    ),
                    location=LocationSchemaName(
                        schema_name= SCHEMA_NAME
                    )
                )
             ).result
    else:
        print("Setting up internal datamart")
        added_data_mart_result = wos_client.data_marts.add(
                background_mode=False,
                name="WOS Data Mart",
                description="Data Mart created by WOS tutorial notebook", 
                internal_database = True).result
        
    data_mart_id = added_data_mart_result.metadata.id
    
else:
    data_mart_id=data_marts[0].metadata.id
    print("Using existing datamart {}".format(data_mart_id))

Using existing datamart 5a0b9076-fcf6-49e8-a824-9e3a6b4c2a56


### Remove existing service provider connected with used  WML instance. 

Multiple service provaders for the same engine instance are avaiable in Watson OpenScale. To avoid multiple service providers of used WML instance in the tutorial notebook the following code deletes existing service provder(s) and then adds new one. 

In [34]:
service_providers = wos_client.service_providers.list().result.service_providers
for service_provider in service_providers:
    service_instance_id = service_provider.entity.instance_id
    if service_instance_id == WML_CREDENTIALS["instance_id"]:
        wos_client.service_providers.delete(service_provider.metadata.id)
        print("Deleted existing service_provider for WML instance", service_instance_id)

Deleted existing service_provider for WML instance fa2ef988-b919-468b-a05b-2a3c9df845d5


## Add service provider

Watson OpenScale needs to be bound to the Watson Machine Learning instance to capture payload data into and out of the model.

**Note:** You can bind more than one engine instance if needed by calling `wos_client.service_providers.add` method. Next, you can refer to particular service provider using `service_provider_id`.

In [35]:
added_service_provider_result = wos_client.service_providers.add(
        name="Watson Machine Learning service provider",
        description="Service Provider added by tutorial WOS notebook",
        service_type=ServiceTypes.WATSON_MACHINE_LEARNING,
        credentials=WMLCredentialsCloud(
            apikey=WML_CREDENTIALS["apikey"],
            url=WML_CREDENTIALS["url"],
            instance_id=WML_CREDENTIALS["instance_id"]
        ),
        background_mode=False
    ).result
service_provider_id = added_service_provider_result.metadata.id




 Waiting for end of adding service provider fed4410c-aa4e-40ab-a0a3-429afc89c638 




active

-----------------------------------------------
 Successfully finished adding service provider 
-----------------------------------------------




In [36]:
wos_client.service_providers.show()

fa2ef988-b919-468b-a05b-2a3c9df845d5,active,Watson Machine Learning service provider,watson_machine_learning,2020-08-18 18:30:40.165000+00:00,fed4410c-aa4e-40ab-a0a3-429afc89c638
6ac7a5f8-c58b-4cf6-9c90-470d6c0a5496,active,New provider,watson_machine_learning,2020-07-29 01:32:16.294000+00:00,6ac7a5f8-c58b-4cf6-9c90-470d6c0a5496


In [37]:
asset_deployment_details = wos_client.service_providers.list_assets(data_mart_id=data_mart_id, service_provider_id=service_provider_id, deployment_id=deployment_uid).result['resources'][0]
print("Get asset details for {asset_name} with id: {id}".format(asset_name=asset_deployment_details['entity']['name'], id = asset_deployment_details['metadata']['guid']))

Get asset details for Scikit German Risk Deployment with id: d9df97da-38aa-4a2f-b4af-4daa0f4e1ff4


## Subscriptions

### Remove existing credit risk subscriptions

In [38]:
wos_client.subscriptions.show()

a2164301-4048-4480-b996-bfb006c95c1b,Implicit Bias v2,5a0b9076-fcf6-49e8-a824-9e3a6b4c2a56,3421e6ce-16c4-478e-84fd-0bdd766fd950,Implicit Bias Deploy v2,6ac7a5f8-c58b-4cf6-9c90-470d6c0a5496,active,2020-08-10 20:44:52.309000+00:00,9afc14f1-2014-4fe2-95a9-fbb57f518e93


This code removes previous subscriptions to the German Credit model to refresh the monitors with the new model and new data.

In [39]:
subscriptions = wos_client.subscriptions.list().result.subscriptions
for subscription in subscriptions:
    sub_model_id = subscription.entity.asset.asset_id
    if sub_model_id == model_uid:
        wos_client.subscriptions.delete(subscription.metadata.id)
        print("Deleted existing subscription for model", model_uid)

This code creates the model subscription in OpenScale using the Python client API. Note that we need to provide the model unique identifier, and some information about the model itself.

In [40]:
subscription_details = wos_client.subscriptions.add(
        data_mart_id=data_mart_id,
        service_provider_id=service_provider_id,
        asset=Asset(
            asset_id=published_model_details["metadata"]["guid"],
            url=published_model_details["metadata"]["url"],
            asset_type=AssetTypes.MODEL,
            input_data_type=InputDataType.STRUCTURED,
            problem_type=ProblemType.BINARY_CLASSIFICATION
        ),
        deployment=AssetDeploymentRequest(
            deployment_id=asset_deployment_details["metadata"]["guid"],
            name=asset_deployment_details["entity"]["name"],
            deployment_type= DeploymentTypes.ONLINE,
            url=asset_deployment_details["metadata"]["url"]
        ),
        asset_properties=AssetPropertiesRequest(
            label_column="Risk",
            probability_fields=["probability"],
            prediction_field="prediction",
            feature_fields = ["CheckingStatus","LoanDuration","CreditHistory","LoanPurpose","LoanAmount","ExistingSavings","EmploymentDuration","InstallmentPercent","Sex","OthersOnLoan","CurrentResidenceDuration","OwnsProperty","Age","InstallmentPlans","Housing","ExistingCreditsCount","Job","Dependents","Telephone","ForeignWorker"],
            categorical_fields = ["CheckingStatus","CreditHistory","LoanPurpose","ExistingSavings","EmploymentDuration","Sex","OthersOnLoan","OwnsProperty","InstallmentPlans","Housing","Job","Telephone","ForeignWorker"],
            training_data_reference=TrainingDataReference(type="cos",
                                                          location=COSTrainingDataReferenceLocation(bucket = BUCKET_NAME,
                                                                                                    file_name = training_data_file_name),
                                                          connection=COSTrainingDataReferenceConnection.from_dict({
                                                                        "resource_instance_id": COS_RESOURCE_CRN,
                                                                        "url": COS_ENDPOINT,
                                                                        "api_key": COS_API_KEY_ID,
                                                                        "iam_url": "https://iam.bluemix.net/oidc/token"})),
            training_data_schema=SparkStruct.from_dict(asset_deployment_details["entity"]["asset_properties"]["training_data_schema"])
        )
    ).result
subscription_id = subscription_details.metadata.id
print(subscription_details)

{
  "metadata": {
    "id": "221f432c-d1d7-4f2a-aa40-94cd6463f546",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/e0b56432b1f1bd804706dc29b8a89ca1:5a0b9076-fcf6-49e8-a824-9e3a6b4c2a56:subscription:221f432c-d1d7-4f2a-aa40-94cd6463f546",
    "url": "/v2/subscriptions/221f432c-d1d7-4f2a-aa40-94cd6463f546",
    "created_at": "2020-08-18T18:30:55.526000Z",
    "created_by": "IBMid-310002F0G1"
  },
  "entity": {
    "data_mart_id": "5a0b9076-fcf6-49e8-a824-9e3a6b4c2a56",
    "service_provider_id": "fed4410c-aa4e-40ab-a0a3-429afc89c638",
    "asset": {
      "asset_id": "c3fbb6ba-1dcb-4aa4-9073-2c0f32d7451c",
      "url": "https://us-south.ml.cloud.ibm.com/v3/wml_instances/fa2ef988-b919-468b-a05b-2a3c9df845d5/published_models/c3fbb6ba-1dcb-4aa4-9073-2c0f32d7451c",
      "asset_type": "model",
      "problem_type": "binary",
      "input_data_type": "structured"
    },
    "asset_properties": {
      "training_data_reference": {
        "secret_id": "c3c887e7-05ae-4de2-9d8b-05db1fdf

In [41]:
import time

time.sleep(5)
payload_data_set_id = None
payload_data_set_id = wos_client.data_sets.list(type=DataSetTypes.PAYLOAD_LOGGING, 
                                                target_target_id=subscription_id, 
                                                target_target_type=TargetTypes.SUBSCRIPTION).result.data_sets[0].metadata.id
if payload_data_set_id is None:
    print("Payload data set not found. Please check subscription status.")
else:
    print("Payload data set id:", payload_data_set_id)

Payload data set id: fa9f83ce-139f-42c8-9fe2-7bb86a9486ae


In [42]:
wos_client.data_sets.show()

5a0b9076-fcf6-49e8-a824-9e3a6b4c2a56,active,221f432c-d1d7-4f2a-aa40-94cd6463f546,subscription,manual_labeling,2020-08-18 18:30:56.365000+00:00,42f5d524-c0b4-4c4f-80e8-c18129dda86e
5a0b9076-fcf6-49e8-a824-9e3a6b4c2a56,active,221f432c-d1d7-4f2a-aa40-94cd6463f546,subscription,payload_logging,2020-08-18 18:30:56.270000+00:00,fa9f83ce-139f-42c8-9fe2-7bb86a9486ae
5a0b9076-fcf6-49e8-a824-9e3a6b4c2a56,active,9afc14f1-2014-4fe2-95a9-fbb57f518e93,subscription,manual_labeling,2020-08-10 20:44:54.177000+00:00,c3dd0628-eb1f-4be8-b29f-9a2321a02223
5a0b9076-fcf6-49e8-a824-9e3a6b4c2a56,active,9afc14f1-2014-4fe2-95a9-fbb57f518e93,subscription,payload_logging,2020-08-10 20:44:54.111000+00:00,980fbdef-3894-47fb-9219-2adfef573c29
5a0b9076-fcf6-49e8-a824-9e3a6b4c2a56,active,5a0b9076-fcf6-49e8-a824-9e3a6b4c2a56,data_mart,explanations_whatif,2020-07-23 15:19:59.475000+00:00,6e8ed0d9-8d45-476a-a1d3-66f6b77be181
5a0b9076-fcf6-49e8-a824-9e3a6b4c2a56,active,5a0b9076-fcf6-49e8-a824-9e3a6b4c2a56,data_mart,explanations,2020-07-23 15:19:59.038000+00:00,b242ee1c-31ba-4473-9e8f-23566edf69da




Get subscription list

In [43]:
wos_client.subscriptions.show()

c3fbb6ba-1dcb-4aa4-9073-2c0f32d7451c,None,5a0b9076-fcf6-49e8-a824-9e3a6b4c2a56,d9df97da-38aa-4a2f-b4af-4daa0f4e1ff4,Scikit German Risk Deployment,fed4410c-aa4e-40ab-a0a3-429afc89c638,active,2020-08-18 18:30:55.526000+00:00,221f432c-d1d7-4f2a-aa40-94cd6463f546
a2164301-4048-4480-b996-bfb006c95c1b,Implicit Bias v2,5a0b9076-fcf6-49e8-a824-9e3a6b4c2a56,3421e6ce-16c4-478e-84fd-0bdd766fd950,Implicit Bias Deploy v2,6ac7a5f8-c58b-4cf6-9c90-470d6c0a5496,active,2020-08-10 20:44:52.309000+00:00,9afc14f1-2014-4fe2-95a9-fbb57f518e93


### Score the model so we can configure monitors

Now that the WML service has been bound and the subscription has been created, we need to send a request to the model before we configure OpenScale. This allows OpenScale to create a payload log in the datamart with the correct schema, so it can capture data coming into and out of the model. First, the code gets the model deployment's endpoint URL, and then sends a few records for predictions.

In [44]:
credit_risk_scoring_endpoint = None
print(deployment_uid)

for deployment in wml_client.deployments.get_details()["resources"]:
    if deployment_uid in deployment["metadata"]["guid"]:
        credit_risk_scoring_endpoint = deployment["entity"]["scoring_url"]
        
print(credit_risk_scoring_endpoint)

d9df97da-38aa-4a2f-b4af-4daa0f4e1ff4
https://us-south.ml.cloud.ibm.com/v3/wml_instances/fa2ef988-b919-468b-a05b-2a3c9df845d5/deployments/d9df97da-38aa-4a2f-b4af-4daa0f4e1ff4/online


In [45]:
fields = ["CheckingStatus","LoanDuration","CreditHistory","LoanPurpose","LoanAmount","ExistingSavings","EmploymentDuration","InstallmentPercent","Sex","OthersOnLoan","CurrentResidenceDuration","OwnsProperty","Age","InstallmentPlans","Housing","ExistingCreditsCount","Job","Dependents","Telephone","ForeignWorker"]
values = [
  ["no_checking",10,"credits_paid_to_date","car_new",1343,"100_to_500","1_to_4",2,"female","none",3,"savings_insurance",46,"none","own",2,"skilled",1,"none","yes"],
  ["no_checking",20,"prior_payments_delayed","furniture",4567,"500_to_1000","1_to_4",4,"male","none",4,"savings_insurance",36,"none","free",2,"management_self-employed",1,"none","yes"],
  ["0_to_200",24,"all_credits_paid_back","car_new",863,"less_100","less_1",2,"female","co-applicant",2,"real_estate",38,"none","own",1,"skilled",1,"none","yes"],
  ["0_to_200",17,"no_credits","car_new",2368,"less_100","1_to_4",3,"female","none",3,"real_estate",29,"none","own",1,"skilled",1,"none","yes"],
  ["0_to_200",66,"no_credits","car_new",250,"less_100","unemployed",2,"female","none",3,"real_estate",23,"none","rent",1,"management_self-employed",1,"none","yes"],
  ["no_checking",14,"credits_paid_to_date","car_new",832,"100_to_500","1_to_4",2,"male","none",2,"real_estate",42,"none","own",1,"skilled",1,"none","yes"],
  ["no_checking",35,"outstanding_credit","appliances",5696,"unknown","greater_7",4,"male","co-applicant",4,"unknown",54,"none","free",2,"skilled",1,"yes","yes"],
  ["0_to_200",23,"prior_payments_delayed","retraining",1375,"100_to_500","4_to_7",3,"male","none",3,"real_estate",37,"none","own",2,"management_self-employed",1,"none","yes"]
]

payload_scoring = {"fields": fields,"values": values}
scoring_response = wml_client.deployments.score(credit_risk_scoring_endpoint, payload_scoring)

print("Single record scoring result:", "\n fields:", scoring_response["fields"], "\n values: ", scoring_response["values"][0])

Single record scoring result: 
 fields: ['prediction', 'probability'] 
 values:  ['No Risk', [0.6270374722208658, 0.3729625277791342]]


In [46]:
time.sleep(5)
pl_records_count = wos_client.data_sets.get_records_count(payload_data_set_id)
print("Number of records in the payload logging table: {}".format(pl_records_count))
if pl_records_count == 0:
    raise Exception("Payload logging did not happen!")

Number of records in the payload logging table: 8


# Quality monitoring and feedback logging <a name="quality"></a>

## Enable quality monitoring

The code below waits ten seconds to allow the payload logging table to be set up before it begins enabling monitors. First, it turns on the quality (accuracy) monitor and sets an alert threshold of 70%. OpenScale will show an alert on the dashboard if the model accuracy measurement (area under the curve, in the case of a binary classifier) falls below this threshold.

The second paramater supplied, min_records, specifies the minimum number of feedback records OpenScale needs before it calculates a new measurement. The quality monitor runs hourly, but the accuracy reading in the dashboard will not change until an additional 50 feedback records have been added, via the user interface, the Python client, or the supplied feedback endpoint.

In [47]:
import time

time.sleep(10)
target = Target(
        target_type=TargetTypes.SUBSCRIPTION,
        target_id=subscription_id
)
parameters = {
    "min_feedback_data_size": 50
}
quality_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.QUALITY.ID,
    target=target,
    parameters=parameters
).result




 Waiting for end of monitor instance creation 5f25c3f7-0547-4aca-bf08-1709f099add6 




active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




In [48]:
quality_monitor_instance_id = quality_monitor_details.metadata.id

## Feedback logging

The code below downloads and stores enough feedback data to meet the minimum threshold so that OpenScale can calculate a new accuracy measurement. It then kicks off the accuracy monitor. The monitors run hourly, or can be initiated via the Python API, the REST API, or the graphical user interface.

In [49]:
!rm additional_feedback_data_v2.json
!wget https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/assets/historical_data/german_credit_risk/wml/additional_feedback_data_v2.json

--2020-08-18 18:31:48--  https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/assets/historical_data/german_credit_risk/wml/additional_feedback_data_v2.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.232.8.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.8.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 50890 (50K) [text/plain]
Saving to: ‘additional_feedback_data_v2.json’

100%[======================================>] 50,890      --.-K/s   in 0.003s  

2020-08-18 18:31:48 (16.8 MB/s) - ‘additional_feedback_data_v2.json’ saved [50890/50890]



### Get feedback logging dataset ID

In [54]:
feedback_dataset_id = None
feedback_dataset = wos_client.data_sets.list(type=DataSetTypes.FEEDBACK, 
                                                target_target_id=subscription_id, 
                                                target_target_type=TargetTypes.SUBSCRIPTION).result
print(feedback_dataset)
feedback_dataset_id = feedback_dataset.data_sets[0].metadata.id
if feedback_dataset_id is None:
    print("Feedback data set not found. Please check quality monitor status.")

{
  "data_sets": [
    {
      "metadata": {
        "id": "99d844ae-972b-4951-9c9f-c137c97653de",
        "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/181ed6cc388f47bd9d862fe066f9cfce:5a0b9076-fcf6-49e8-a824-9e3a6b4c2a56:data_set:99d844ae-972b-4951-9c9f-c137c97653de",
        "url": "/v2/data_sets/99d844ae-972b-4951-9c9f-c137c97653de",
        "created_at": "2020-08-18T18:31:38.983000Z",
        "created_by": "iam-ServiceId-2e5c9fda-38bf-4279-9712-cdb3b6f3a7ad",
        "modified_at": "2020-08-18T18:31:39.178000Z",
        "modified_by": "iam-ServiceId-2e5c9fda-38bf-4279-9712-cdb3b6f3a7ad"
      },
      "entity": {
        "data_mart_id": "5a0b9076-fcf6-49e8-a824-9e3a6b4c2a56",
        "name": "221f432c-d1d7-4f2a-aa40-94cd6463f546_feedback",
        "description": "221f432c-d1d7-4f2a-aa40-94cd6463f546_feedback",
        "type": "feedback",
        "target": {
          "target_type": "subscription",
          "target_id": "221f432c-d1d7-4f2a-aa40-94cd6463f546"
        },
    

In [55]:
with open('additional_feedback_data_v2.json') as feedback_file:
    additional_feedback_data = json.load(feedback_file)

In [56]:
wos_client.data_sets.store_records(feedback_dataset_id, request_body=additional_feedback_data, background_mode=False)




 Waiting for end of storing records with request id: 76f8ede6-26cf-48c4-a2bc-e3c5975fabc6 




active

---------------------------------------
 Successfully finished storing records 
---------------------------------------




In [57]:
wos_client.data_sets.get_records_count(data_set_id=feedback_dataset_id)

98

In [58]:
run_details = wos_client.monitor_instances.run(monitor_instance_id=quality_monitor_instance_id, background_mode=False).result




 Waiting for end of monitoring run dd2c67a9-cbce-43b2-b274-e5e09957de36 




running
finished

---------------------------
 Successfully finished run 
---------------------------




In [59]:
wos_client.monitor_instances.show_metrics(monitor_instance_id=quality_monitor_instance_id)

2020-08-18 18:34:14.125000+00:00,true_positive_rate,d795e44d-4387-4b95-8ef1-a7e69e996f5a,0.5151515151515151,0.8,None,['model_type:original'],quality,5f25c3f7-0547-4aca-bf08-1709f099add6,dd2c67a9-cbce-43b2-b274-e5e09957de36,subscription,221f432c-d1d7-4f2a-aa40-94cd6463f546
2020-08-18 18:34:14.125000+00:00,area_under_roc,d795e44d-4387-4b95-8ef1-a7e69e996f5a,0.7191142191142192,0.8,None,['model_type:original'],quality,5f25c3f7-0547-4aca-bf08-1709f099add6,dd2c67a9-cbce-43b2-b274-e5e09957de36,subscription,221f432c-d1d7-4f2a-aa40-94cd6463f546
2020-08-18 18:34:14.125000+00:00,precision,d795e44d-4387-4b95-8ef1-a7e69e996f5a,0.7727272727272727,0.8,None,['model_type:original'],quality,5f25c3f7-0547-4aca-bf08-1709f099add6,dd2c67a9-cbce-43b2-b274-e5e09957de36,subscription,221f432c-d1d7-4f2a-aa40-94cd6463f546
2020-08-18 18:34:14.125000+00:00,f1_measure,d795e44d-4387-4b95-8ef1-a7e69e996f5a,0.6181818181818182,0.8,None,['model_type:original'],quality,5f25c3f7-0547-4aca-bf08-1709f099add6,dd2c67a9-cbce-43b2-b274-e5e09957de36,subscription,221f432c-d1d7-4f2a-aa40-94cd6463f546
2020-08-18 18:34:14.125000+00:00,accuracy,d795e44d-4387-4b95-8ef1-a7e69e996f5a,0.7857142857142857,0.8,None,['model_type:original'],quality,5f25c3f7-0547-4aca-bf08-1709f099add6,dd2c67a9-cbce-43b2-b274-e5e09957de36,subscription,221f432c-d1d7-4f2a-aa40-94cd6463f546
2020-08-18 18:34:14.125000+00:00,log_loss,d795e44d-4387-4b95-8ef1-a7e69e996f5a,0.43364420900703426,None,0.8,['model_type:original'],quality,5f25c3f7-0547-4aca-bf08-1709f099add6,dd2c67a9-cbce-43b2-b274-e5e09957de36,subscription,221f432c-d1d7-4f2a-aa40-94cd6463f546
2020-08-18 18:34:14.125000+00:00,false_positive_rate,d795e44d-4387-4b95-8ef1-a7e69e996f5a,0.07692307692307693,None,0.8,['model_type:original'],quality,5f25c3f7-0547-4aca-bf08-1709f099add6,dd2c67a9-cbce-43b2-b274-e5e09957de36,subscription,221f432c-d1d7-4f2a-aa40-94cd6463f546
2020-08-18 18:34:14.125000+00:00,area_under_pr,d795e44d-4387-4b95-8ef1-a7e69e996f5a,0.6670321020970371,0.8,None,['model_type:original'],quality,5f25c3f7-0547-4aca-bf08-1709f099add6,dd2c67a9-cbce-43b2-b274-e5e09957de36,subscription,221f432c-d1d7-4f2a-aa40-94cd6463f546
2020-08-18 18:34:14.125000+00:00,recall,d795e44d-4387-4b95-8ef1-a7e69e996f5a,0.5151515151515151,0.8,None,['model_type:original'],quality,5f25c3f7-0547-4aca-bf08-1709f099add6,dd2c67a9-cbce-43b2-b274-e5e09957de36,subscription,221f432c-d1d7-4f2a-aa40-94cd6463f546


# Fairness monitoring, Drift and explanations <a name="fairness"></a>

### Fairness configuration

The code below configures fairness monitoring for our model. It turns on monitoring for two features, Sex and Age. In each case, we must specify:
  * Which model feature to monitor
  * One or more **majority** groups, which are values of that feature that we expect to receive a higher percentage of favorable outcomes
  * One or more **minority** groups, which are values of that feature that we expect to receive a higher percentage of unfavorable outcomes
  * The threshold at which we would like OpenScale to display an alert if the fairness measurement falls below (in this case, 95%)

Additionally, we must specify which outcomes from the model are favourable outcomes, and which are unfavourable. We must also provide the number of records OpenScale will use to calculate the fairness score. In this case, OpenScale's fairness monitor will run hourly, but will not calculate a new fairness rating until at least 200 records have been added. Finally, to calculate fairness, OpenScale must perform some calculations on the training data, so we provide the dataframe containing the data.

In [60]:
wos_client.monitor_instances.show()

5a0b9076-fcf6-49e8-a824-9e3a6b4c2a56,active,221f432c-d1d7-4f2a-aa40-94cd6463f546,subscription,quality,2020-08-18 18:31:38.519000+00:00,5f25c3f7-0547-4aca-bf08-1709f099add6
5a0b9076-fcf6-49e8-a824-9e3a6b4c2a56,active,9afc14f1-2014-4fe2-95a9-fbb57f518e93,subscription,fairness,2020-08-10 20:45:13.815000+00:00,7b0e3d9a-1d77-4430-b824-cacca7a65216
5a0b9076-fcf6-49e8-a824-9e3a6b4c2a56,active,9afc14f1-2014-4fe2-95a9-fbb57f518e93,subscription,performance,2020-08-10 20:44:55.164000+00:00,920c5f4a-1bff-49aa-9a75-b07cdc946b9d


In [61]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id

)
parameters = {
    "features": [
        {"feature": "Sex",
         "majority": ['male'],
         "minority": ['female'],
         "threshold": 0.95
         },
        {"feature": "Age",
         "majority": [[26, 75]],
         "minority": [[18, 25]],
         "threshold": 0.95
         }
    ],
    "favourable_class": ["No Risk"],
    "unfavourable_class": ["Risk"],
    "min_records": 4
}

fairness_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.FAIRNESS.ID,
    target=target,
    parameters=parameters).result
fairness_monitor_instance_id =fairness_monitor_details.metadata.id




 Waiting for end of monitor instance creation bd566ef4-5f66-42c4-aacd-08eeb99a182a 




active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




### Drift configuration

In [62]:
monitor_instances = wos_client.monitor_instances.list().result.monitor_instances
for monitor_instance in monitor_instances:
    monitor_def_id=monitor_instance.entity.monitor_definition_id
    if monitor_def_id == "drift" and monitor_instance.entity.target.target_id == subscription_id:
        wos_client.monitor_instances.delete(monitor_instance.metadata.id)
        print('Deleted existing drift monitor instance with id: ', monitor_instance.metadata.id)


target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id

)
parameters = {
    "min_samples": 20,
    "train_drift_model": True,
    "enable_model_drift": False,
    "enable_data_drift": True
}

drift_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.DRIFT.ID,
    target=target,
    parameters=parameters
).result

drift_monitor_instance_id = drift_monitor_details.metadata.id




 Waiting for end of monitor instance creation e577101c-311c-4d28-bb24-e92544db3e47 




preparing
active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




## Score the model again now that monitoring is configured

This next section randomly selects 200 records from the data feed and sends those records to the model for predictions. This is enough to exceed the minimum threshold for records set in the previous section, which allows OpenScale to begin calculating fairness.

In [63]:
from IPython.utils import io
!rm german_credit_feed.json
with io.capture_output() as captured:
    !wget https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/assets/historical_data/german_credit_risk/wml/german_credit_feed.json -O german_credit_feed.json
!ls -lh german_credit_feed.json

-rw-r----- 1 dsxuser dsxuser 3.0M Aug 18 18:34 german_credit_feed.json


In [64]:
scoring_url

'https://us-south.ml.cloud.ibm.com/v3/wml_instances/fa2ef988-b919-468b-a05b-2a3c9df845d5/deployments/d9df97da-38aa-4a2f-b4af-4daa0f4e1ff4/online'

Score 200 randomly chosen records

In [65]:
import random

with open('german_credit_feed.json', 'r') as scoring_file:
    scoring_data = json.load(scoring_file)

fields = scoring_data['fields']
values = []
for _ in range(200):
    values.append(random.choice(scoring_data['values']))
payload_scoring = {"fields": fields, "values": values}

scoring_response = wml_client.deployments.score(credit_risk_scoring_endpoint, payload_scoring)
time.sleep(5)

**Note:** Now in payload table should be total 208 records.

In [66]:
print("Number of records in payload table:", wos_client.data_sets.get_records_count(data_set_id=payload_data_set_id))

Number of records in payload table: 208


## Run fairness monitor

Kick off a fairness monitor run on current data. The monitor runs hourly, but can be manually initiated using the Python client, the REST API, or the graphical user interface.

In [67]:
run_details = wos_client.monitor_instances.run(monitor_instance_id=fairness_monitor_instance_id, background_mode=False)




 Waiting for end of monitoring run f6cbf330-e4f1-4175-ab11-41d8f197250b 




running
finished

---------------------------
 Successfully finished run 
---------------------------




In [68]:
time.sleep(10)

wos_client.monitor_instances.show_metrics(monitor_instance_id=fairness_monitor_instance_id)

2020-08-18 18:35:10.710720+00:00,fairness_value,dbc586c4-880a-46a8-a637-91c4899e0c6a,100.0,80.0,None,"['feature:Sex', 'fairness_metric_type:fairness', 'feature_value:female']",fairness,bd566ef4-5f66-42c4-aacd-08eeb99a182a,f6cbf330-e4f1-4175-ab11-41d8f197250b,subscription,221f432c-d1d7-4f2a-aa40-94cd6463f546
2020-08-18 18:35:10.710720+00:00,fairness_value,dbc586c4-880a-46a8-a637-91c4899e0c6a,100.0,80.0,None,"['feature:Age', 'fairness_metric_type:fairness', 'feature_value:18-25']",fairness,bd566ef4-5f66-42c4-aacd-08eeb99a182a,f6cbf330-e4f1-4175-ab11-41d8f197250b,subscription,221f432c-d1d7-4f2a-aa40-94cd6463f546
2020-08-18 18:35:10.710720+00:00,fairness_value,9c5cb71b-cb73-45aa-a914-2e3666ee8db4,100.0,80.0,None,"['feature:Age', 'fairness_metric_type:debiased_fairness', 'feature_value:18-25']",fairness,bd566ef4-5f66-42c4-aacd-08eeb99a182a,f6cbf330-e4f1-4175-ab11-41d8f197250b,subscription,221f432c-d1d7-4f2a-aa40-94cd6463f546
2020-08-18 18:35:10.710720+00:00,fairness_value,9c5cb71b-cb73-45aa-a914-2e3666ee8db4,100.0,80.0,None,"['feature:Sex', 'fairness_metric_type:debiased_fairness', 'feature_value:female']",fairness,bd566ef4-5f66-42c4-aacd-08eeb99a182a,f6cbf330-e4f1-4175-ab11-41d8f197250b,subscription,221f432c-d1d7-4f2a-aa40-94cd6463f546
2020-08-18 18:34:38.773571+00:00,fairness_value,18c34df7-e61a-470a-b998-3a7154691acb,100.0,80.0,None,"['feature:Sex', 'fairness_metric_type:fairness', 'feature_value:female']",fairness,bd566ef4-5f66-42c4-aacd-08eeb99a182a,4a491bae-874d-4b8d-bf35-59c1d5721622,subscription,221f432c-d1d7-4f2a-aa40-94cd6463f546
2020-08-18 18:34:38.773571+00:00,fairness_value,18c34df7-e61a-470a-b998-3a7154691acb,100.0,80.0,None,"['feature:Age', 'fairness_metric_type:fairness', 'feature_value:18-25']",fairness,bd566ef4-5f66-42c4-aacd-08eeb99a182a,4a491bae-874d-4b8d-bf35-59c1d5721622,subscription,221f432c-d1d7-4f2a-aa40-94cd6463f546
2020-08-18 18:34:38.773571+00:00,fairness_value,0c7e0551-9606-4b53-b36a-db4e63a2e236,100.0,80.0,None,"['feature:Sex', 'fairness_metric_type:debiased_fairness', 'feature_value:female']",fairness,bd566ef4-5f66-42c4-aacd-08eeb99a182a,4a491bae-874d-4b8d-bf35-59c1d5721622,subscription,221f432c-d1d7-4f2a-aa40-94cd6463f546
2020-08-18 18:34:38.773571+00:00,fairness_value,0c7e0551-9606-4b53-b36a-db4e63a2e236,100.0,80.0,None,"['feature:Age', 'fairness_metric_type:debiased_fairness', 'feature_value:18-25']",fairness,bd566ef4-5f66-42c4-aacd-08eeb99a182a,4a491bae-874d-4b8d-bf35-59c1d5721622,subscription,221f432c-d1d7-4f2a-aa40-94cd6463f546


## Run drift monitor


Kick off a drift monitor run on current data. The monitor runs every hour, but can be manually initiated using the Python client, the REST API.

In [69]:
drift_run_details = wos_client.monitor_instances.run(monitor_instance_id=drift_monitor_instance_id, background_mode=False)




 Waiting for end of monitoring run 621e8896-702a-4b29-9660-3ca1ae67891f 




finished

---------------------------
 Successfully finished run 
---------------------------




In [70]:
time.sleep(5)

wos_client.monitor_instances.show_metrics(monitor_instance_id=drift_monitor_instance_id)

2020-08-18 18:35:32.545296+00:00,data_drift_magnitude,7fd256cf-1e66-4106-80ba-c484ee507fc0,0.11538461538461539,None,None,[],drift,e577101c-311c-4d28-bb24-e92544db3e47,621e8896-702a-4b29-9660-3ca1ae67891f,subscription,221f432c-d1d7-4f2a-aa40-94cd6463f546


## Configure Explainability

Finally, we provide OpenScale with the training data to enable and configure the explainability features.

In [71]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)
parameters = {
    "enabled": True
}
explainability_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.EXPLAINABILITY.ID,
    target=target,
    parameters=parameters
).result

explainability_monitor_id = explainability_details.metadata.id




 Waiting for end of monitor instance creation abf29d95-c6c0-4c6a-8f5a-22a0cd912f34 




active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




## Run explanation for sample record

In [72]:
pl_records_resp = wos_client.data_sets.get_list_of_records(data_set_id=payload_data_set_id, limit=1, offset=0).result
scoring_ids = [pl_records_resp["records"][0]["entity"]["values"]["scoring_id"]]
print("Running explanations on scoring IDs: {}".format(scoring_ids))
explanation_types = ["lime", "contrastive"]
result = wos_client.monitor_instances.explanation_tasks(scoring_ids=scoring_ids, explanation_types=explanation_types).result
print(result)


Running explanations on scoring IDs: ['db4e076775f5ac2fbfe97e25da96d0f5-1']
{
  "metadata": {
    "explanation_task_ids": [
      "9ce1439e-5bb7-435c-83d3-e0f96293b2ad"
    ],
    "created_by": "IBMid-310002F0G1",
    "created_at": "2020-08-18T18:35:58.197415Z"
  }
}


# Custom monitors and metrics <a name="custom"></a>

## Register custom monitor

In [70]:
def get_definition(monitor_name):
    monitor_definitions = wos_client.monitor_definitions.list().result.monitor_definitions
    
    for definition in monitor_definitions:
        if monitor_name == definition.entity.name:
            return definition
    
    return None

In [71]:
monitor_name = 'my model performance'
metrics = [MonitorMetricRequest(name='sensitivity',
                                thresholds=[MetricThreshold(type=MetricThresholdTypes.LOWER_LIMIT, default=0.8)]),
          MonitorMetricRequest(name='specificity',
                                thresholds=[MetricThreshold(type=MetricThresholdTypes.LOWER_LIMIT, default=0.75)])]
tags = [MonitorTagRequest(name='region', description='customer geographical region')]

existing_definition = get_definition(monitor_name)

if existing_definition is None:
    custom_monitor_details = wos_client.monitor_definitions.add(name=monitor_name, metrics=metrics, tags=tags, background_mode=False).result
else:
    custom_monitor_details = existing_definition




 Waiting for end of adding monitor definition my_model_performance 




finished

-------------------------------------------------
 Successfully finished adding monitor definition 
-------------------------------------------------




### Show available monitors types

In [72]:
wos_client.monitor_definitions.show()

my_model_performance,my model performance,"['sensitivity', 'specificity']"
assurance,Assurance,"['Uncertainty', 'Confidence']"
fairness,Fairness,"['Fairness value', 'Average Odds Difference metric value', 'False Discovery Rate Difference metric value', 'Error Rate Difference metric value', 'False Negative Rate Difference metric value', 'False Omission Rate Difference metric value', 'False Positive Rate Difference metric value', 'True Positive Rate Difference metric value']"
performance,Performance,['Number of records']
explainability,Explainability,[]
mrm,Model risk management monitoring,"['Tests run', 'Tests passed', 'Tests failed', 'Tests skipped', 'Fairness score', 'Quality score', 'Drift score']"
correlations,Correlations,"['Maximum positive correlation coefficient', 'Maximum negative correlation coefficient', 'Mean absolute correlation coefficient', 'Significant correlation coefficients count']"
drift,Drift,"['Drop in accuracy', 'Predicted accuracy', 'Drop in data consistency']"
quality,Quality,"['Area under ROC', 'Area under PR', 'Proportion explained variance', 'Mean absolute error', 'Mean squared error', 'R squared', 'Root of mean squared error', 'Accuracy', 'Weighted True Positive Rate (wTPR)', 'True positive rate (TPR)', 'Weighted False Positive Rate (wFPR)', 'False positive rate (FPR)', 'Weighted recall', 'Recall', 'Weighted precision', 'Precision', 'Weighted F1-Measure', 'F1-Measure', 'Logarithmic loss']"


### Get monitors uids and details

In [73]:
custom_monitor_id = custom_monitor_details.metadata.id

print(custom_monitor_id)

my_model_performance


In [74]:
custom_monitor_details = wos_client.monitor_definitions.get(monitor_definition_id=custom_monitor_id).result
print('Monitor definition details:', custom_monitor_details)

Monitor definition details: {
  "metadata": {
    "id": "my_model_performance",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/d2c5c1830d2a6fc88d77c610c09bb1ed:9c72f0c0-6d24-4503-9fce-b31695f6c931:monitor_definition:my_model_performance",
    "url": "/v2/monitor_definitions/my_model_performance",
    "created_at": "2020-07-20T10:29:32.562000Z",
    "created_by": "IBMid-50P0BSQW1S"
  },
  "entity": {
    "name": "my model performance",
    "metrics": [
      {
        "name": "sensitivity",
        "thresholds": [
          {
            "type": "lower_limit",
            "default": 0.8
          }
        ],
        "expected_direction": "increasing",
        "id": "sensitivity"
      },
      {
        "name": "specificity",
        "thresholds": [
          {
            "type": "lower_limit",
            "default": 0.75
          }
        ],
        "expected_direction": "increasing",
        "id": "specificity"
      }
    ],
    "tags": [
      {
        "name": "region

## Enable custom monitor for subscription

In [75]:
target = Target(
        target_type=TargetTypes.SUBSCRIPTION,
        target_id=subscription_id
    )

thresholds = [MetricThresholdOverride(metric_id='sensitivity', type = MetricThresholdTypes.LOWER_LIMIT, value=0.9)]

custom_monitor_instance_details = wos_client.monitor_instances.create(
            data_mart_id=data_mart_id,
            background_mode=False,
            monitor_definition_id=custom_monitor_id,
            target=target
).result




 Waiting for end of monitor instance creation f0ac28dd-a67b-41dc-8f57-fae9ce2a506d 




active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




### Get monitor instance id and configuration details

In [76]:
custom_monitor_instance_id = custom_monitor_instance_details.metadata.id

In [77]:
custom_monitor_instance_details = wos_client.monitor_instances.get(custom_monitor_instance_id).result
print(custom_monitor_instance_details)

{
  "metadata": {
    "id": "f0ac28dd-a67b-41dc-8f57-fae9ce2a506d",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/d2c5c1830d2a6fc88d77c610c09bb1ed:9c72f0c0-6d24-4503-9fce-b31695f6c931:monitor_instance:f0ac28dd-a67b-41dc-8f57-fae9ce2a506d",
    "url": "/v2/monitor_instances/f0ac28dd-a67b-41dc-8f57-fae9ce2a506d",
    "created_at": "2020-07-20T10:29:50.961000Z",
    "created_by": "IBMid-50P0BSQW1S"
  },
  "entity": {
    "data_mart_id": "9c72f0c0-6d24-4503-9fce-b31695f6c931",
    "monitor_definition_id": "my_model_performance",
    "target": {
      "target_type": "subscription",
      "target_id": "3d54d760-80f9-41a3-b746-fc23ba3f978e"
    },
    "thresholds": [
      {
        "metric_id": "sensitivity",
        "type": "lower_limit",
        "value": 0.8
      },
      {
        "metric_id": "specificity",
        "type": "lower_limit",
        "value": 0.75
      }
    ],
    "schedule": {
      "repeat_interval": 60,
      "repeat_unit": "minute",
      "repeat_type": "min

## Storing custom metrics

In [78]:
from datetime import datetime, timezone, timedelta
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import MonitorMeasurementRequest
custom_monitoring_run_id = "11122223333111abc"
measurement_request = [MonitorMeasurementRequest(timestamp=datetime.now(timezone.utc), 
                                                 metrics=[{"specificity": 0.78, "sensitivity": 0.67, "region": "us-south"}], run_id=custom_monitoring_run_id)]
print(measurement_request[0])

{
  "timestamp": "2020-07-20T10:29:59.519529Z",
  "run_id": "11122223333111abc",
  "metrics": [
    {
      "specificity": 0.78,
      "sensitivity": 0.67,
      "region": "us-south"
    }
  ]
}


In [79]:
published_measurement_response = wos_client.monitor_instances.measurements.add(
    monitor_instance_id=custom_monitor_instance_id,
    monitor_measurement_request=measurement_request).result
published_measurement_id = published_measurement_response[0]["measurement_id"]
print(published_measurement_response)

[{'measurement_id': '17af7d72-95c2-4793-96d0-07e07393383d', 'metrics': [{'region': 'us-south', 'sensitivity': 0.67, 'specificity': 0.78}], 'run_id': '11122223333111abc', 'timestamp': '2020-07-20T10:29:59.519529Z'}]


### List and get custom metrics

In [80]:
time.sleep(5)
published_measurement = wos_client.monitor_instances.measurements.get(monitor_instance_id=custom_monitor_instance_id, measurement_id=published_measurement_id).result
print(published_measurement)

{
  "metadata": {
    "id": "17af7d72-95c2-4793-96d0-07e07393383d",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/d2c5c1830d2a6fc88d77c610c09bb1ed:9c72f0c0-6d24-4503-9fce-b31695f6c931:measurement:17af7d72-95c2-4793-96d0-07e07393383d",
    "url": "/v2/monitor_instances/f0ac28dd-a67b-41dc-8f57-fae9ce2a506d/measurements/17af7d72-95c2-4793-96d0-07e07393383d",
    "created_at": "2020-07-20T10:30:01.247000Z",
    "created_by": "IBMid-50P0BSQW1S"
  },
  "entity": {
    "timestamp": "2020-07-20T10:29:59.519529Z",
    "run_id": "11122223333111abc",
    "values": [
      {
        "metrics": [
          {
            "id": "sensitivity",
            "value": 0.67,
            "lower_limit": 0.8
          },
          {
            "id": "specificity",
            "value": 0.78,
            "lower_limit": 0.75
          }
        ],
        "tags": [
          {
            "id": "region",
            "value": "us-south"
          }
        ]
      }
    ],
    "issue_count": 1,
    "t

## Insert historical fairness metrics

In [75]:
!rm history_fairness_v2.json
with io.capture_output() as captured:
    !wget https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/assets/historical_data/german_credit_risk/wml/history_fairness_v2.json
!ls -lh history_fairness_v2.json

rm: cannot remove ‘history_fairness_v2.json’: No such file or directory
-rw-r----- 1 dsxuser dsxuser 37K Aug 18 18:39 history_fairness_v2.json


In [73]:
historyDays = 7

In [76]:
from datetime import datetime, timedelta, timezone

with open('history_fairness_v2.json', 'r') as history_file:
    payloads = json.load(history_file)

for day in range(historyDays):
    print('Loading day', day + 1)
    daily_measurement_requests = []
    
    for hour in range(24):
        score_time = datetime.now(timezone.utc) + timedelta(hours=(-(24*day + hour + 1)))
        index = (day * 24 + hour) % len(payloads) # wrap around and reuse values if needed
 
        measurement_request = MonitorMeasurementRequest(timestamp=score_time,metrics = [payloads[index][0], payloads[index][1]])
        daily_measurement_requests.append(measurement_request)
        
        
    response = wos_client.monitor_instances.measurements.add(
                                            monitor_instance_id=fairness_monitor_instance_id,
                                            monitor_measurement_request=daily_measurement_requests).result     
print('Finished')

Loading day 1
Loading day 2
Loading day 3
Loading day 4
Loading day 5
Loading day 6
Loading day 7
Finished


## Insert historical debias metrics

In [77]:
!rm history_debias_v2.json
with io.capture_output() as captured:
    !wget https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/assets/historical_data/german_credit_risk/wml/history_debias_v2.json
!ls -lh history_debias_v2.json

rm: cannot remove ‘history_debias_v2.json’: No such file or directory
-rw-r----- 1 dsxuser dsxuser 37K Aug 18 18:39 history_debias_v2.json


In [78]:
with open('history_debias_v2.json', 'r') as history_file:
    payloads = json.load(history_file)

for day in range(historyDays):
    print('Loading day', day + 1)
    daily_measurement_requests = []
    for hour in range(24):
        score_time = datetime.now(timezone.utc) + timedelta(hours=(-(24*day + hour + 1)))
        index = (day * 24 + hour) % len(payloads) # wrap around and reuse values if needed

        measurement_request = MonitorMeasurementRequest(timestamp=score_time,metrics = [payloads[index][0], payloads[index][1]])
        
        daily_measurement_requests.append(measurement_request)
        
    response = wos_client.monitor_instances.measurements.add(
                                            monitor_instance_id=fairness_monitor_instance_id,
                                            monitor_measurement_request=daily_measurement_requests).result     

print('Finished')

Loading day 1
Loading day 2
Loading day 3
Loading day 4
Loading day 5
Loading day 6
Loading day 7
Finished


## Insert historical quality metrics

In [79]:
measurements = [0.76, 0.78, 0.68, 0.72, 0.73, 0.77, 0.80]
for day in range(historyDays):
    quality_measurement_requests = []
    print('Loading day', day + 1)
    for hour in range(24):
        score_time = datetime.utcnow() + timedelta(hours=(-(24*day + hour + 1)))
        score_time = score_time.isoformat() + "Z"
        
        metric = {"area_under_roc": measurements[day]}
                
        measurement_request = MonitorMeasurementRequest(timestamp=score_time,metrics = [metric])
        quality_measurement_requests.append(measurement_request)
        
        
    response = wos_client.monitor_instances.measurements.add(
                                            monitor_instance_id=quality_monitor_instance_id,
                                            monitor_measurement_request=quality_measurement_requests).result    
    
print('Finished')

Loading day 1
Loading day 2
Loading day 3
Loading day 4
Loading day 5
Loading day 6
Loading day 7
Finished


## Insert historical confusion matrixes

In [80]:
!rm history_quality_metrics.json
with io.capture_output() as captured:
    !wget https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/assets/historical_data/german_credit_risk/wml/history_quality_metrics.json
!ls -lh history_quality_metrics.json

rm: cannot remove ‘history_quality_metrics.json’: No such file or directory
-rw-r----- 1 dsxuser dsxuser 79K Aug 18 18:39 history_quality_metrics.json


In [81]:
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import Source

with open('history_quality_metrics.json') as json_file:
    records = json.load(json_file)
    
for day in range(historyDays):
    index = 0
    cm_measurement_requests = []
    print('Loading day', day + 1)
    
    for hour in range(24):
        score_time = datetime.utcnow() + timedelta(hours=(-(24*day + hour + 1)))
        score_time = score_time.isoformat() + "Z"

        metric = records[index]['metrics']
        source = records[index]['sources']

        
        measurement_request = {"timestamp": score_time, "metrics": [metric], "sources": [source]}
        cm_measurement_requests.append(measurement_request)

        index+=1

    response = wos_client.monitor_instances.measurements.add(monitor_instance_id=quality_monitor_instance_id, monitor_measurement_request=cm_measurement_requests).result    

print('Finished')

Loading day 1
Loading day 2
Loading day 3
Loading day 4
Loading day 5
Loading day 6
Loading day 7
Finished


## Insert historical performance metrics

In [82]:
target = Target(
        target_type=TargetTypes.INSTANCE,
        target_id=payload_data_set_id
    )


performance_monitor_instance_details = wos_client.monitor_instances.create(
            data_mart_id=data_mart_id,
            background_mode=False,
            monitor_definition_id=wos_client.monitor_definitions.MONITORS.PERFORMANCE.ID,
            target=target
).result
performance_monitor_instance_id = performance_monitor_instance_details.metadata.id





 Waiting for end of monitor instance creation 84c994f6-132a-4ea0-9794-4ffbdeda5e1c 




active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




In [83]:
for day in range(historyDays):
    performance_measurement_requests = []
    print('Loading day', day + 1)
    for hour in range(24):
        score_time = datetime.utcnow() + timedelta(hours=(-(24*day + hour + 1)))
        score_time = score_time.isoformat() + "Z"
        score_count = random.randint(60, 600)
        
        metric = {"record_count": score_count, "data_set_type": "scoring_payload"}
        
        measurement_request = {"timestamp": score_time, "metrics": [metric]}
        performance_measurement_requests.append(measurement_request)
        
    response = wos_client.monitor_instances.measurements.add(
                                            monitor_instance_id=performance_monitor_instance_id,
                                            monitor_measurement_request=performance_measurement_requests).result    

print('Finished')

Loading day 1
Loading day 2
Loading day 3
Loading day 4
Loading day 5
Loading day 6
Loading day 7
Finished


## Insert historical drift measurements

In [84]:
!rm history_drift_measurement_0.json
with io.capture_output() as captured_0:
    !wget https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/assets/historical_data/german_credit_risk/wos/history_drift_measurement_0.json
!rm history_drift_measurement_1.json
with io.capture_output() as captured_1:
    !wget https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/assets/historical_data/german_credit_risk/wos/history_drift_measurement_1.json
!rm history_drift_measurement_2.json
with io.capture_output() as captured_2:
    !wget https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/assets/historical_data/german_credit_risk/wos/history_drift_measurement_2.json
!rm history_drift_measurement_3.json
with io.capture_output() as captured_3:
    !wget https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/assets/historical_data/german_credit_risk/wos/history_drift_measurement_3.json
!rm history_drift_measurement_4.json
with io.capture_output() as captured_4:
    !wget https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/assets/historical_data/german_credit_risk/wos/history_drift_measurement_4.json
!rm history_drift_measurement_5.json
with io.capture_output() as captured_5:
    !wget https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/assets/historical_data/german_credit_risk/wos/history_drift_measurement_5.json
!rm history_drift_measurement_6.json
with io.capture_output() as captured_6:
    !wget https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/assets/historical_data/german_credit_risk/wos/history_drift_measurement_6.json
!ls -lh history_drift_measurement_0.json

rm: cannot remove ‘history_drift_measurement_0.json’: No such file or directory
rm: cannot remove ‘history_drift_measurement_1.json’: No such file or directory
rm: cannot remove ‘history_drift_measurement_2.json’: No such file or directory
rm: cannot remove ‘history_drift_measurement_3.json’: No such file or directory
rm: cannot remove ‘history_drift_measurement_4.json’: No such file or directory
rm: cannot remove ‘history_drift_measurement_5.json’: No such file or directory
rm: cannot remove ‘history_drift_measurement_6.json’: No such file or directory
-rw-r----- 1 dsxuser dsxuser 832K Aug 18 18:40 history_drift_measurement_0.json


In [85]:
for day in range(historyDays):
    drift_measurements = []

    with open("history_drift_measurement_{}.json".format(day), 'r') as history_file:
        drift_daily_measurements = json.load(history_file)
    print('Loading day', day + 1)

    #Historical data contains 8 records per day - each represents 3 hour drift window.
    
    for nb_window, records in enumerate(drift_daily_measurements):
        for record in records:
            window_start =  datetime.utcnow() + timedelta(hours=(-(24 * day + (nb_window+1)*3 + 1))) # first_payload_record_timestamp_in_window (oldest)
            window_end = datetime.utcnow() + timedelta(hours=(-(24 * day + nb_window*3 + 1)))# last_payload_record_timestamp_in_window (most recent)
            #modify start and end time for each record
            record['sources'][0]['data']['start'] = window_start.isoformat() + "Z"
            record['sources'][0]['data']['end'] = window_end.isoformat() + "Z"
            
            
            metric = record['metrics'][0]
            source = record['sources'][0]

            measurement_request = {"timestamp": window_start.isoformat() + "Z", "metrics": [metric], "sources": [source]}
            
            drift_measurements.append(measurement_request)
        
    response = wos_client.monitor_instances.measurements.add(
                                            monitor_instance_id=drift_monitor_instance_id,
                                            monitor_measurement_request=drift_measurements).result    

    
    print("Daily loading finished.")

Loading day 1
Daily loading finished.
Loading day 2
Daily loading finished.
Loading day 3
Daily loading finished.
Loading day 4
Daily loading finished.
Loading day 5
Daily loading finished.
Loading day 6
Daily loading finished.
Loading day 7
Daily loading finished.


## Additional data to help debugging

In [86]:
print('Datamart:', data_mart_id)
print('Model:', model_uid)
print('Deployment:', deployment_uid)
print('Scoring URL:', credit_risk_scoring_endpoint)

Datamart: 5a0b9076-fcf6-49e8-a824-9e3a6b4c2a56
Model: c3fbb6ba-1dcb-4aa4-9073-2c0f32d7451c
Deployment: d9df97da-38aa-4a2f-b4af-4daa0f4e1ff4
Scoring URL: https://us-south.ml.cloud.ibm.com/v3/wml_instances/fa2ef988-b919-468b-a05b-2a3c9df845d5/deployments/d9df97da-38aa-4a2f-b4af-4daa0f4e1ff4/online


## Identify transactions for Explainability

Transaction IDs identified by the cells below can be copied and pasted into the Explainability tab of the OpenScale dashboard.

In [93]:
wos_client.data_sets.show_records(payload_data_set_id, limit=5)

c5df6bf4-dbe0-4a75-8896-89e18e62f4d0,"[0.9623357543734388, 0.03766424562656128]",1344,ac2d1d427658e5881c5f8b3a8f361c69-1,3,less_100,2020-07-20T10:28:16.119712Z,0.9623357543734388,None,less_0,business,No Risk,unemployed,bank,none,25,19,prior_payments_delayed,2,yes,own,1,55a4c90d-916f-4339-8a23-9ba18fbe70e7,none,male,1,car_other,1_to_4,No Risk,"[0.9623357543734388, 0.03766424562656128]"
c5df6bf4-dbe0-4a75-8896-89e18e62f4d0,"[0.46661053524400775, 0.5333894647559922]",4920,ac2d1d427658e5881c5f8b3a8f361c69-10,4,greater_1000,2020-07-20T10:28:16.119712Z,0.5333894647559922,None,no_checking,furniture,Risk,skilled,none,none,26,46,prior_payments_delayed,3,yes,own,3,55a4c90d-916f-4339-8a23-9ba18fbe70e7,yes,male,2,savings_insurance,4_to_7,Risk,"[0.46661053524400775, 0.5333894647559922]"
c5df6bf4-dbe0-4a75-8896-89e18e62f4d0,"[0.7004603633710788, 0.29953963662892125]",6486,ac2d1d427658e5881c5f8b3a8f361c69-100,3,500_to_1000,2020-07-20T10:28:16.119712Z,0.7004603633710788,None,no_checking,repairs,No Risk,unemployed,bank,none,44,53,prior_payments_delayed,4,yes,rent,1,55a4c90d-916f-4339-8a23-9ba18fbe70e7,none,female,1,car_other,1_to_4,No Risk,"[0.7004603633710788, 0.29953963662892125]"
c5df6bf4-dbe0-4a75-8896-89e18e62f4d0,"[0.7109586027107817, 0.28904139728921824]",3516,ac2d1d427658e5881c5f8b3a8f361c69-101,4,less_100,2020-07-20T10:28:16.119712Z,0.7109586027107817,None,0_to_200,furniture,No Risk,skilled,none,none,28,33,credits_paid_to_date,3,yes,own,2,55a4c90d-916f-4339-8a23-9ba18fbe70e7,none,female,1,savings_insurance,4_to_7,No Risk,"[0.7109586027107817, 0.28904139728921824]"
c5df6bf4-dbe0-4a75-8896-89e18e62f4d0,"[0.9832902168115192, 0.01670978318848079]",250,ac2d1d427658e5881c5f8b3a8f361c69-102,1,less_100,2020-07-20T10:28:16.119712Z,0.9832902168115192,None,0_to_200,car_new,No Risk,skilled,none,none,8,19,all_credits_paid_back,2,yes,rent,1,55a4c90d-916f-4339-8a23-9ba18fbe70e7,none,male,1,real_estate,less_1,No Risk,"[0.9832902168115192, 0.01670978318848079]"


## Congratulations!

You have finished the hands-on lab for IBM Watson OpenScale. You can now view the [OpenScale Dashboard](https://aiopenscale.cloud.ibm.com/). Click on the tile for the German Credit model to see fairness, accuracy, and performance monitors. Click on the timeseries graph to get detailed information on transactions during a specific time window.

## Next steps

OpenScale shows model performance over time. You have two options to keep data flowing to your OpenScale graphs:
  * Download, configure and schedule the [model feed notebook](https://raw.githubusercontent.com/emartensibm/german-credit/master/german_credit_scoring_feed.ipynb). This notebook can be set up with your WML credentials, and scheduled to provide a consistent flow of scoring requests to your model, which will appear in your OpenScale monitors.
  * Re-run this notebook. Running this notebook from the beginning will delete and re-create the model and deployment, and re-create the historical data. Please note that the payload and measurement logs for the previous deployment will continue to be stored in your datamart, and can be deleted if necessary.